# Witness rankについて

参考：
* [Model-based RL in Contextual Decision Processes: PAC bounds and Exponential Improvements over Model-free Approaches](https://arxiv.org/abs/1811.08540)

[Bellman rank](RL_Bellman_rank.ipynb)の続きです．
Witness rankはモデルベースRL向けのBellman rankみたいなものです．
モデルベースとモデルフリーでは一般に，
* モデルベース：環境のダイナミクスをモデル化し，それを解くことで最適方策を見つける　
* モデルフリー：最適方策やその価値を直接モデル化する

といった違いがあります．
モデルベースはモデルフリーよりも一般にサンプル効率が良いことが信じられています（教師ありの情報をたくさん使えるので）．

TODO: globalな探索が重要な環境ではモデルフリーでしか解けないことがBellman rankの論文で指摘されているっぽい．要確認．

今回は，関数近似が十分リッチであれば，モデルベースな手法がモデルフリーよりもサンプル効率が指数的に良いことを示していきます．

表記：

Contextual decision process (CDP)を考えます．

* $\mathcal{X}$を$\mathcal{X}_1, \dots, \mathcal{X}_{H+1}$に分割可能な文脈の空間とします．
* $|\mathcal{A}|=K$とします．

---

Contextual MDPの重要な例として，Factored MDPがあります．
$d\in \mathbb{N}$と$\mathcal{O}$を小さな有限集合として，文脈空間を
$\mathcal{X} = [H] \times \mathcal{O}^d$とします．

また，$x \in \mathcal{X}$について，$x[i]$を$i \in [d]$番目の状態における文脈の値とします．
このとき，それぞれの状態変数$i \in [d]$と，そのparent  $\operatorname{pa}_i \subseteq [d]$を考えましょう．

Factored MDPでは，

$$
...
$$

の形式で遷移確率が分解されます．このとき，遷移の作用素は
$\sum^{d}_{i=1} HK \cdot |\mathcal{O}^{1+|\operatorname{pa}_i|}|$個のパラメータで構成されます．
一方で，通常のMDPでは$d H K |\mathcal{O}|^{1+d}$のパラメータがかかることに注意しましょう（TODO: なんで$d$があるの？）．
よって，factored MDPの方が遷移に必要なパラメータの数が少ないです．

---

* モデルベースでは報酬と遷移のモデル$M=(R, P)$を$M \in \mathcal{M}$から選択して，最も良いものを見つけるとします．
    * $(r, x') \sim M_{x, a}$は$r\sim R(x, a)$かつ$x' \sim P_{x, a}$として表記します
    * $\operatorname{OP}(M)$で，モデル$M$についての最適価値と最適方策を計算するオラクルを表すとします．

## モデルベースとモデルフリーの定義

よく使われるモデルフリーの定義は，「メモリのスペースが$o\left(|\mathcal{X}|^2|\mathcal{A}|\right)$であるアルゴリズム」ですが，
これは関数近似がある場合にスケールしづらいです．

そこで，次の定義を使いましょう．

---

（有限な）関数集合$\mathcal{G}: (\mathcal{X} \times \mathcal{A}) \to \mathbb{R}$について，$\mathcal{G}$-profileを$\Phi_{\mathcal{G}}(x):=[g(x, a)]_{g \in \mathcal{G}, a \in \mathcal{A}}$として定義します．

次を満たす時，アルゴリズムは$\mathcal{G}$を使ったモデルフリーアルゴリズムといいます．

全ての$x\in \mathcal{X}$に，$\Phi_{\mathcal{G}}(x)$を使ってアクセスする．

---

つまり，モデルフリーでは$x$と$x'$の違いを，$\Phi_{\mathcal{G}}(x)$を使ってでしか確認できません．
（例えば$x$と$x'$の違いを$Q(x, \cdot)$と$Q(x', \cdot)$を使って判別する，など）

そのため，一般には何らかの情報理論的な損失が発生します．
（ちなみにモデルベースとモデルフリーアルゴリズムは，Tabular MDPにおいては，情報理論的には同じになります（Appendix D）．）

関数近似がある場合では，一方で，モデルフリーでは解けない場合が存在します．

---

**定理**

次を満たすCDPの族$\mathcal{M}$が存在します．

* $|\mathcal{M}|\leq 2^H$
* 確率$1- \delta$以上で，$\mathcal{M}$を使ったモデルベースアルゴリズムが$\epsilon$-最適な方策を$\operatorname{poly}(H, 1 / \epsilon, \log (1 / \delta))$個の軌跡で吐き出す
* $\mathcal{G}=O P(\mathcal{M})$として，どんなモデルフリーアルゴリズムも$o(2^H)$の軌跡を使っても$\epsilon$-最適方策が吐き出せない 

---

実際，factored MDPにおいてモデルふr−アルゴリズムは上手く行かないみたいですね（lower boundはまだ出されてないみたいですが）

## Witness model misfit

Integral probability metricを使って次を定義します．

---

**Integral probability metric**

Wasserstein距離の一般化みたいなもんです．

$P_1, P_2 \in \Delta(\mathcal{Z})$ over $z \in \mathcal{Z}$と，Symmetric（つまり$f\in \mathcal{F}$なら$-f\in \mathcal{F}$）な関数の集合$\mathcal{F}: \mathcal{Z} \rightarrow \mathbb{R}$について，

$$
\sup _{f \in \mathcal{F}} \mathbb{E}_{z \sim P_1}[f(z)]-\mathbb{E}_{z \sim P_2}[f(z)]
$$

のことを$\mathcal{F}$のIPMと呼ぶ．

---

**Witness model misfit**

次の

* クラス：$\mathcal{F}: \mathcal{X} \times \mathcal{A} \times \mathcal{R} \times \mathcal{X} \rightarrow \mathbb{R}$：$(x, a, r, x')$を受け取って何か実数を返す関数．例えばベルマン誤差など．
* モデル：$M, M^{\prime} \in \mathcal{M}$
* タイムステップ：$h\in [H]$

について，$M'$が$M$にwitnessされたときの$h$におけるWitnessed model misfitは

$$
\mathcal{W}\left(M, M^{\prime}, h ; \mathcal{F}\right) \triangleq \sup _{f \in \mathcal{F}} \underset{\substack{x_h \sim \pi_M \\ a_h \sim \pi_{M^{\prime}}}}{\mathbb{E}}\left[\underset{\left(r, x^{\prime}\right) \sim M_h^{\prime}}{\mathbb{E}}\left[f\left(x_h, a_h, r, x^{\prime}\right)\right]-\underset{\left(r, x^{\prime}\right) \sim M_h^{\star}}{\mathbb{E}}\left[f\left(x_h, a_h, r, x^{\prime}\right)\right]\right],
$$

で定義されます．ここで，モデル$M=(R, P)$について，$\left(r, x^{\prime}\right) \sim M_h$ は $r \sim R_{x_h, a_h}, x^{\prime} \sim P_{x_h, a_h}$の略記です．

---

**直感**

* misfitは，$M'$と$M^*$によって生じる２つの分布のIPMを測っています．
    * この分布は$\mathcal{X} \times \mathcal{A} \times \mathcal{R} \times \mathcal{X}$上に定義されます．
* もちろん$M_h'=M_h^*$ならmisfitは０になります．
* $M$はデータを集めるようのモデルです．[RL_Bellman_rank.ipynb](RL_Bellman_rank.ipynb)と同じノリ？

例えば$\mathcal{F}=\left\{f:\|f\|_{\infty} \leq 1\right\}$であれば，misfitは

$$\mathcal{W}\left(M, M^{\prime}, h ; \mathcal{F}\right)=\mathbb{E}\left[\left\|R_{x_h, a_h}^{\prime} \circ P_{x_h, a_h}^{\prime}-R_{x_h, a_h}^{\star} \circ P_{x_h, a_h}^{\star}\right\|_{T V} \mid x_h \sim \pi_M, a_h \sim \pi_{M^{\prime}}\right]$$

になります（ここで$R_{x, a} \circ P_{x, a}$は$\mathcal{R}\times \mathcal{X}$上の分布です）．これは単に$R^{\prime} \circ P^{\prime}$ and $R^{\star} \circ P^{\star}$間のTV距離を測ってるだけです．

なので，直感的には$\mathcal{F}$によってIPMの測り方が決まります．

---

**Average Bellman Errorとの関係**

Bellman rankのAverage Bellman Errorを思い出しましょう．Average Bellman Errorは次で定義されます：

$$
\mathcal{E}_B\left(Q, Q^{\prime}, h\right) \triangleq \mathbb{E}\left[Q^{\prime}\left(x_h, a_h\right)-r_h-Q^{\prime}\left(x_{h+1}, a_{h+1}\right) \mid x_h \sim \pi_Q, a_{h: h+1} \sim \pi_{Q^{\prime}}\right]
$$

ここで，Q関数が$\mathcal{Q}=\mathrm{OP}(\mathcal{M})$の形をしている場合，上の定義を２つのモデル$M, M'$に対する誤差として拡張できます．
そのとき，$M, M'$に対するAverage Bellman Errorは関数$f_{M^{\prime}}\left(x, a, r, x^{\prime}\right)=r+V_{M^{\prime}}\left(x^{\prime}\right)$によってwitnessされたmisfitと同じです．

---

$\mathcal{F}$は何でも良いわけではないです．次を仮定します：
（クラス$\mathcal{F}: \mathcal{Z}\to \mathbb{R}$は$f \in \mathcal{F}$なら$-f \in \mathcal{F}$のときに対象であるといいます．）

$\mathcal{F}$は対象であり，有限なサイズとします．
$\|f \|_\infty \leq 2$とし，
$$
\forall M, M^{\prime} \in \mathcal{M}: \mathcal{W}\left(M, M^{\prime}, h ; \mathcal{F}\right) \geq \mathcal{E}_B\left(Q_M, Q_{M^{\prime}}, h\right)
$$
が成立するとします．つまり，

$$
\begin{aligned}
&\sup _{f \in \mathcal{F}} \underset{\substack{x_h \sim \pi_M \\ a_h \sim \pi_{M^{\prime}}}}{\mathbb{E}}\left[\underset{\left(r, x^{\prime}\right) \sim M_h^{\prime}}{\mathbb{E}}\left[f\left(x_h, a_h, r, x^{\prime}\right)\right]-\underset{\left(r, x^{\prime}\right) \sim M_h^{\star}}{\mathbb{E}}\left[f\left(x_h, a_h, r, x^{\prime}\right)\right]\right]
\\\geq & 
\mathbb{E}\left[Q_{M'}\left(x_h, a_h\right)-r_h-Q_{M^{\prime}}\left(x_{h+1}, a_{h+1}\right) \mid x_h \sim \pi_Q, a_{h: h+1} \sim \pi_{Q_{M'}}\right]
\end{aligned}
$$

を仮定します．

**仮定についての注意**

* この仮定は，$r(x, a)+V_M(x') \in \mathcal{F}$である場合に常に成立します．実際，この場合はMisfitが平均ベルマン誤差に一致します．ただし，必ずしもこの場合を含む必要はありません．

---


## Witness rank

これまではWitness model misfitを定義し，仮定を一つ導入しました．実はこれだけでは十分ではありません．指数的なサンプルが必要になるMDPが存在してしまいます（論文のProposition 4）．

そこで，Witness rankの仮定を導入します．ノリはBellman rankと同じです．

---

**Witness rank**

モデルクラス$\mathcal{M}$, テスト関数$\mathcal{F}$，そして$\kappa \in (0, 1]$を考えます．
それぞれの$h \in [H]$に対して，行列の集合$\mathcal{N}_{\kappa, h}$を次のように定めます．
任意の$A \in \mathcal{N}_{\kappa, h}$が次を満たす：

$$
A \in \mathbb{R}^{|\mathcal{M}| \times|\mathcal{M}|}, \quad \kappa \mathcal{E}_B\left(M, M^{\prime}, h\right) \leq A\left(M, M^{\prime}\right) \leq \mathcal{W}\left(M, M^{\prime}, h\right), \forall M, M^{\prime} \in \mathcal{M},
$$

これについて，Witness rankを

$$
\mathrm{W}(\kappa, \beta, \mathcal{M}, \mathcal{F}, h) \triangleq \min _{A \in \mathcal{N}_{\kappa, h}} \operatorname{rank}(A, \beta)
$$

として定義します．

**解釈**

* $A(M, M')={W}\left(M, M^{\prime}, h\right)$である場合を考えましょう．
この時，$0$でないwitnessed model misfitをチェックするためにかかる文脈の分布の数はWitness rankで抑えられます．よって，$M^\star$を見つけるためにかかる時間はwitness rankで抑えられます．
* 一方で，$A\left(M, M^{\prime}\right)=\kappa \mathcal{E}_B\left(M, M^{\prime}, h\right)$の場合はこれはBellman rankと同じです．よって，上のWitness rankの定義はBellman rankを一般化してることがわかります．


## アルゴリズム

Model misfitを推定して最適方策を出すアルゴリズムを見てみましょう．

データセット$\mathcal{D}=\{(x_h^{(n)}, a_h^{(n)}, r_h^{(n)}, x_{h+1}^{(n)})\}_{n=1}^N$が与えられたときを考えます．ここで，

$$
x_h^{(n)} \sim \pi_M, a_h^{(n)} \sim U(\mathcal{A}), (r_h^{(n)}, x_{h+1}^{(n)}) \sim M_h^\star
$$

です．また，importance weightを$\rho^{(n)} = K\pi_{M'}(a_h^{(n)} \mid x_h^{(n)})$とします．

つまり，
* 各ステップ$h$までは$\pi_M$で動き，$x_h^{(n)}$を出す
* その次に行動$a_h^{(n)}$を$U(\mathcal{A})$から出す
* 報酬と次状態は真のMDPからサンプルする

状況です．これを使ってmisfitを推定してみましょう．

$$
\widehat{\mathcal{W}}(M, M', h) = \max_{f \in \mathcal{F}}
\sum^{N}_{n=1}\frac{\rho^{(n)}}{N}
\mathbb{E}_{(r, x')\sim M'_h}
\left[f(x_h^{(n)}, a_h^{(n)}, r, x') - f(x_h^{(n)}, a_h^{(n)}, r^{(n)}_h, x^{(n)}_{h+1})\right]
$$

ここで，行動は$U(\mathcal{A})$からサンプルしてるので，重み$\rho^{n}$で修正しています．

これは通常の一様収束の話を使えば$\mathcal{W}(M, M', h)$に収束することが言えます．

また，平均ベルマン誤差$\mathcal{E}_B(M, M, h)$も推定します．
データセット
$\{(x_h^{(n)}, a_h^{(n)}, r_h^{(n)}, x_{h+1}^{(n)})\}_{n=1}^N$
を考えます．ここで，
$$
x_h^{(n)} \sim \pi_M, a_h^{(n)} \sim \pi_M, (r_h^{(n)}, x_{h+1}^{(n)}) \sim M_h^\star
$$
を使って，unbiasedな推定
$$
\widehat{\mathcal{E}}_B(M, M, h) = \frac{1}{N} \sum^N_{n=1} 
\left[Q_M(x^{(n)}_h, a^{(n)}_h) - \left[r_h^{(n)} + V_M(x_{h+1}^{(n)})\right]\right]
$$
をします．

以上を踏まえて，次のアルゴリズムを考えます：

![witness-rank](figs/Witness-rank.png)


このアルゴリズムでは，ループ毎に$\mathcal{M}_t$から最適なモデル以外が除去されていきます．

* ３行目ではOptimisticなモデルを選択し，４行目でその価値をサンプルから推定してます．
* ４，５行目で，その推定値がOptimisticなモデルの価値とあまり変わらない場合，その方策をそのまま返します．
* ６行目で，平均ベルマン誤差が高いステップを探します．
* ７行目で，このステップの精度を上げます
* 最後にWitness misfitが一定以上のモデルを排除します．$M^\star$のmisfitは０なので